In [4]:
%pip install torch transformers safetensors numpy


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from typing import Tuple
import warnings

In [6]:
class RealTimeTextDeepfakeDetector:
    def __init__(self, model_path: str = "model.safetensors"):
        """
        Initialize with safe model loading
        
        Args:
            model_path: Path to .safetensors or .pt model file
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        
        # Load model architecture
        self.model = BertForSequenceClassification.from_pretrained(
            'bert-base-uncased',
            num_labels=2
        ).to(self.device)
        
        # Safe model loading
        try:
            if model_path.endswith('.safetensors'):
                from safetensors.torch import load_file
                state_dict = load_file(model_path, device=str(self.device))
            else:
                # For .pt files with safe loading
                state_dict = torch.load(model_path, map_location=self.device, weights_only=True)
            
            self.model.load_state_dict(state_dict)
        except Exception as e:
            warnings.warn(f"Safe loading failed, trying legacy load: {str(e)}")
            state_dict = torch.load(model_path, map_location=self.device, weights_only=False)
            self.model.load_state_dict(state_dict)
        
        self.model.eval()
        self.threshold = 0.85
        self._warm_up()

    def _warm_up(self):
        """Initialize model with dummy inference"""
        self.predict("Model warmup")

    def predict(self, text: str) -> Tuple[str, float]:
        """
        Make prediction on input text
        
        Args:
            text: Input string to analyze
            
        Returns:
            tuple: (prediction, confidence)
        """
        inputs = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=512,
            return_tensors='pt'
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)[0].cpu().numpy()
        
        pred = np.argmax(probs)
        confidence = float(np.max(probs))
        
        if pred == 1 and confidence >= self.threshold:
            return "Fake", confidence
        return "Real", 1 - confidence if pred == 1 else confidence


In [7]:
# Example Usage
if __name__ == "__main__":
    try:
        detector = RealTimeTextDeepfakeDetector("/Users/fenilvadher/Documents/Collage Data/SEM - 6/AI/AI Project/bert_fake_news_model/model.safetensors")
        
        print("Real-time Deepfake Detector (Type 'exit' to quit)")
        while True:
            text = input("\nEnter text: ")
            if text.lower() in ('exit', 'quit'):
                break
                
            pred, conf = detector.predict(text)
            print(f"Result: {pred} (Confidence: {conf:.1%})")
            
    except Exception as e:
        print(f"Error: {str(e)}")
        print("Possible solutions:")
        print("1. Convert your model to .safetensors format")
        print("2. Use torch.save(model.state_dict()) for .pt files")
        print("3. Install safetensors: pip install safetensors")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Real-time Deepfake Detector (Type 'exit' to quit)
Result: Fake (Confidence: 99.9%)
Result: Fake (Confidence: 99.9%)
Result: Real (Confidence: 100.0%)
Result: Fake (Confidence: 98.9%)
Result: Fake (Confidence: 98.8%)
Result: Real (Confidence: 100.0%)
Result: Fake (Confidence: 99.9%)
Result: Fake (Confidence: 99.9%)
